In [ ]:
import cv2
import time
import numpy as np
import Handtrackingmodels as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
#####################################################
Wcam, hCam = 640, 480
#####################################################
cap = cv2.VideoCapture(0)
cap.set(3, Wcam)
cap.set(4, hCam)
pTime = 0

detector = htm.handDetector(detectionCon=0.7,maxHands=1)



devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()

minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0



while True:
    success, img = cap.read()
    detector.findHands(img) # detects the hands
    lmList = detector.findPosition(img, draw=False) # finds the positions of the hands
    if len(lmList) !=0:
        # print(lmList[4], lmList[8])

        x1 , y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx,cy = (x1+x2)//2, (y1+y2)//2  #finding center
        
        cv2.circle(img, (x1,y1), 10, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2,y2), 10, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1,y1),(x2,y2),(255,0,255),3)
        cv2.circle(img, (cx,cy), 15, (255, 0, 255), cv2.FILLED)

        length = math.hypot(x2-x1,y2-y2)   #length of the line
        # hand - length 4 - 60
        # Volume Range -65 - 0

        vol = np.interp(length, [4,68], [minVol,maxVol])
        volBar = np.interp(length, [4,68], [400,150])
        volPer = np.interp(length, [4,68], [0,100])
        print(vol)
        volume.SetMasterVolumeLevel(vol, None)
        if length < 4:
            cv2.circle(img, (cx,cy), 15, (0, 255, 0), cv2.FILLED)
    cv2.rectangle(img, (50, 150), (85, 400), (0, 255, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (0, 255, 0), cv2.FILLED)
        # print(length)
    cTime = time.time()
    fps =  1/(cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}',(40,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)
    cv2.putText(img, f' {int(volPer)} %',(40,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0,),3)
    cv2.imshow('Image',img)
    cv2.waitKey(1)

